In [1]:
import pandas as pd
import numpy as np
import re

In [10]:
import os
import urllib.request
from tqdm import tqdm_notebook

In [11]:
def save_img(df):
    num_pic = len(df['사진url'])
    
    # image 폴더 없으면 생성
    if not os.path.exists('actors_img'):
        os.makedirs('actors_img')
    
    for i in tqdm_notebook(range(0, num_pic)):
        try:
            url = df['사진url'][i]
            name = df['Name'][i]
            urllib.request.urlretrieve(url, f"actors_img/{name}.jpg")
        except Exception as e:
            print(f"{i} {name}에서 에러 발생")
            print(e)
            pass

In [4]:
actors_df = pd.read_csv('test_actors_info.csv', encoding='utf-8-sig')
actors_df.head()

,Name,Age,Sex,수상내역,드라마활동,activation_content,사진url
0,감우성,52,남,"['2018 SBS 연기대상 대상', '2018 SBS 연기대상 베스트 커플상', ...",['바람이 분다/주연/권도훈 역/https://search.naver.com/sea...,"MBC 20기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규, 박철, 차광수,...",https://search.pstatic.net/common?type=b&size=...
1,강경준,39,남,"['2017 MBC 연기대상 연속극부문 남자 우수연기상', '2004 MBC 방송연...",['별별 며느리/주연/최한주 역/https://search.naver.com/sea...,"초등학교 시절에는 야구, 중학교 시절에는 농구선수였었다. 농구를 소재로 한 드라마 ...",https://search.pstatic.net/common?type=b&size=...
2,강기영,39,남,"['2019 제8회 대한민국 베스트 스타상 베스트 조연상', '2019 제12회 코...",['이상한 변호사 우영우/주연/정명석 역/https://search.naver.co...,"tvN 드라마 고교처세왕, 오 나의 귀신님에서 감초역할로 등장하며 이름을 알렸다. ...",https://search.pstatic.net/common?type=b&size=...
3,강남길,64,남,['2009 MBC 연기대상 황금연기상 중견배우 부문'],['왼손잡이 아내/조연/오창수 역/https://search.naver.com/se...,"서울수색초등학교, 서울양정중학교, 서울대신고등학교를 졸업했고 한양대학교 영어영문학과...",https://search.pstatic.net/common?type=b&size=...
4,강지섭,41,남,['2006 SBS 연기대상 뉴스타상'],['브라보 마이 라이프/주연/설도현 역/https://search.naver.com...,고향이 부산이며 70만원을 들고 서울로 상경해 고시원에서 생활을 했다. 어렸을 때부...,https://search.pstatic.net/common?type=b&size=...


## 1. 이미지 추출

In [12]:
save_img(actors_df)

C:\Users\USER\AppData\Local\Temp\ipykernel_5332\1428063913.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(0, num_pic)):


  0%|          | 0/256 [00:00<?, ?it/s]

## 2. 드라마활동 > 배역별 내용 추출

In [46]:
import requests
from bs4 import BeautifulSoup

In [52]:
resp = requests.get('https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&qvt=0&query=%EB%B0%94%EB%9E%8C%EC%9D%B4%EB%B6%84%EB%8B%A4%EA%B6%8C%EB%8F%84%ED%9B%88')
soup = BeautifulSoup(resp.text)
soup.select_one('.desc._text').text

'첫사랑 수진과 열렬한 연애 끝에 결혼까지 성공했다. 남에게 피해 주지 않고 올곧고 바르게 살자는 모토 하나로 아름다운 인생을 누려왔다. 아이를 좋아해 제과업체 제품개발팀에서 인생을 올인했고, 하루빨리 안정적이고 여유로운 상황을 만들어 수진과 행복하게 살날만을 바랬다. 탄탄대로인 도훈에게 어두운 그림자가 덮쳤다. 하지만, 어느새 사랑과 가족의 중요함을 까맣게 잊어버린 남자.'

In [62]:
def extract_casting_content(df):
    c_df = df.copy()
    acting_info_list = []
    
    for i in tqdm_notebook(c_df.드라마활동):
        acting_info = []
        
        #drama_activation_list = re.findall("'([^']*)'", i)
        drama_acting_content_url_list =  re.findall("https([^']*)'", i)
        
        for url in drama_acting_content_url_list:
            resp = requests.get('https' + url)
            soup = BeautifulSoup(resp.text)
            try:
                drama_acting_content = soup.select_one('.desc._text').text
            except:
                drama_acting_content = ''
            acting_info.append(drama_acting_content)
        
        acting_info_list.append(acting_info)
    
    c_df['배역내용'] = acting_info_list
    
    return c_df

In [63]:
tmp = extract_casting_content(actors_df)

C:\Users\USER\AppData\Local\Temp\ipykernel_5332\2826311475.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(c_df.드라마활동):


  0%|          | 0/256 [00:00<?, ?it/s]

In [64]:
tmp.head()

,Name,Age,Sex,수상내역,드라마활동,activation_content,사진url,배역내용
0,감우성,52,남,"['2018 SBS 연기대상 대상', '2018 SBS 연기대상 베스트 커플상', ...",['바람이 분다/주연/권도훈 역/https://search.naver.com/sea...,"MBC 20기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규, 박철, 차광수,...",https://search.pstatic.net/common?type=b&size=...,[첫사랑 수진과 열렬한 연애 끝에 결혼까지 성공했다. 남에게 피해 주지 않고 올곧고...
1,강경준,39,남,"['2017 MBC 연기대상 연속극부문 남자 우수연기상', '2004 MBC 방송연...",['별별 며느리/주연/최한주 역/https://search.naver.com/sea...,"초등학교 시절에는 야구, 중학교 시절에는 농구선수였었다. 농구를 소재로 한 드라마 ...",https://search.pstatic.net/common?type=b&size=...,"[흙수저? 아니, 몸짱 얼짱 마음까지 짱인 태권도장 사범. 얼굴만 봐도 신뢰감이 뚝..."
2,강기영,39,남,"['2019 제8회 대한민국 베스트 스타상 베스트 조연상', '2019 제12회 코...",['이상한 변호사 우영우/주연/정명석 역/https://search.naver.co...,"tvN 드라마 고교처세왕, 오 나의 귀신님에서 감초역할로 등장하며 이름을 알렸다. ...",https://search.pstatic.net/common?type=b&size=...,"[일에 대한 자부심이 강한 법무법인 한바다의 시니어 변호사이자 우영우의 멘토., 2..."
3,강남길,64,남,['2009 MBC 연기대상 황금연기상 중견배우 부문'],['왼손잡이 아내/조연/오창수 역/https://search.naver.com/se...,"서울수색초등학교, 서울양정중학교, 서울대신고등학교를 졸업했고 한양대학교 영어영문학과...",https://search.pstatic.net/common?type=b&size=...,[산하의 아버지. 천성이 무골호인으로 허허실실 부드럽지만 처자식을 사랑하는 데는 세...
4,강지섭,41,남,['2006 SBS 연기대상 뉴스타상'],['브라보 마이 라이프/주연/설도현 역/https://search.naver.com...,고향이 부산이며 70만원을 들고 서울로 상경해 고시원에서 생활을 했다. 어렸을 때부...,https://search.pstatic.net/common?type=b&size=...,"[안하무인의 절대 강자. 드라마, 영화, CF에서 몸값 높은 배우이다. 현장에서 갑..."


## 3. Age 중 이상 값 처리

In [65]:
tmp.Age.unique()

array([  52,   39,   64,   41,   53,   40,   49,   25,   57,   42,   50,
         82,   32,   60,   44,   34,   35,   69,   76,   68,   51,   88,
         20,   28,   37,   43,   29,   77,   27,   31,   75,   78,   36,
         80,   73,   33,   67,   58,   46,   85,   48,   30,   45,   83,
         66,   56, 1978,   79,   38,   70,   61,   26,   55,   62,   81,
         71,   65,   47,   15,   63,   54], dtype=int64)

In [67]:
tmp[tmp.Age == 1978]

Int64Index([79], dtype='int64')

In [69]:
tmp.loc[79, 'Age'] = 2022 - 1978

,Name,Age,Sex,수상내역,드라마활동,activation_content,사진url,배역내용


In [70]:
tmp.Age.unique()

array([52, 39, 64, 41, 53, 40, 49, 25, 57, 42, 50, 82, 32, 60, 44, 34, 35,
       69, 76, 68, 51, 88, 20, 28, 37, 43, 29, 77, 27, 31, 75, 78, 36, 80,
       73, 33, 67, 58, 46, 85, 48, 30, 45, 83, 66, 56, 79, 38, 70, 61, 26,
       55, 62, 81, 71, 65, 47, 15, 63, 54], dtype=int64)

## 4. 저장

In [71]:
tmp.to_csv('clean_test_actors_info.csv', encoding='utf-8-sig', index=False)